# Анализ уровня потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России #

<br>Задача:

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

<br>План:
1. Подключиться к базе данных
2. Выгрузить данные
3. Построить дашборд в Tableau, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 
4. Ответить на вопросы с помощью дашборда:
   - Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
   - Какие группы пользователей наиболее лояльны к сервису? Какие менее?
   - Какой общий NPS среди всех опрошенных?
   - Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
5. Подготовить презентацию
    
</div>

In [3]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [5]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [7]:
query = """
SELECT u.user_id,
       u.lt_day as lifetime,
       CASE 
           WHEN u.lt_day <= 365 THEN 'новый'
           ELSE 'старый'
           END AS is_new,
       age,
       CASE
           WHEN u.gender_segment == 0 THEN 'мужчина'
           WHEN u.gender_segment == 1 THEN 'женщина'
           ELSE 'пропуск'
           END AS gender,
       LOWER(u.os_name) AS os_name,
       LOWER(u.cpe_type_name) AS cpe_type,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'сторонник'
           WHEN u.nps_score >= 7 THEN 'нейтрал'
           ELSE 'критик'
           END AS nps_type
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""



In [9]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lifetime,is_new,age,gender,os_name,cpe_type,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_type
0,A001A2,2320,старый,45.0,женщина,android,smartphone,Россия,Уфа,45-54,1-5,36+,10,сторонник
1,A001WF,2344,старый,53.0,мужчина,android,smartphone,Россия,Киров,45-54,1-5,36+,10,сторонник
2,A003Q7,467,старый,57.0,мужчина,android,smartphone,Россия,Москва,55-64,20-25,13-24,10,сторонник
3,A004TB,4190,старый,44.0,женщина,ios,smartphone,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонник
4,A004XT,1163,старый,24.0,мужчина,android,smartphone,Россия,Рязань,16-24,5-10,36+,10,сторонник


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lifetime          502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender            502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type          502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_type          502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


https://public.tableau.com/app/profile/george.veviorski/viz/NPSprojectone/NPSproject